In [1]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
import seaborn as sns 
from sklearn.metrics import mean_squared_error
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
#matplotlib.rcParams['text.color'] = 'G'
df = pd.read_excel("прогноз.xlsx")

In [ ]:
sns.set(rc={'figure.figsize':(11.7,5.27)})
sns.lineplot(x = "Атрибут",
             y = "Значение",
             hue = "Оператор",
             data = df,
             palette=['red', 'yellow', 'green', 'black']);

Нулевая гипотеза расширенного теста Дики-Фуллера (ADF) заключается в том, что временной ряд не является стационарным. Таким образом, если p-значение теста ADF меньше уровня значимости (0,05), то мы отвергнем нулевую гипотезу и сделаем вывод, что временной ряд определенно стационарен. Как мы видим, p-значение более значимо, чем уровень значимости. если данные стационарны - преобразование бокса кокса


In [3]:
# проверка является ли ряд стационарным

from statsmodels.tsa.stattools import adfuller 
from numpy import log 
import pandas as pd 
 
mydata = pd.read_excel('прогноз.xlsx', names = ['value'], header = 0) 
 
res = adfuller( mydata.value.dropna()) 
print('Augmented Dickey-Fuller Statistic: %f' % res[0]) 
print('p-value: %f' % res[1])


Augmented Dickey-Fuller Statistic: -1.053193
p-value: 0.733373


In [27]:
#import numpy as np
#y = np.log(y)
#y

In [ ]:
y = df.set_index(["Атрибут"])
y.head(5)

In [5]:
y_t2 = df[df['Оператор']=='Tele2'].set_index(["Атрибут"])
y_t2.drop('Оператор', axis= 1 , inplace= True ) 

#y_mts = df.query("Оператор == 'МТС'").set_index(["Атрибут"])
#y_mts.drop('Оператор', axis= 1 , inplace= True )

#y_bee = df.loc[df['Оператор'] == 'Билайн'].set_index(["Атрибут"])
#y_bee.drop('Оператор', axis= 1 , inplace= True )

#y_mf = df[df.Оператор.str.startswith('Мегафон')].set_index(["Атрибут"])
#y_mf.drop('Оператор', axis= 1 , inplace= True )



In [ ]:
#трендб шум сезон
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8
decomposition = sm.tsa.seasonal_decompose(y_t2, model='additive')
#decomposition1 = sm.tsa.seasonal_decompose(y_mts, model='additive')
#decomposition2 = sm.tsa.seasonal_decompose(y_bee, model='additive')
#decomposition3 = sm.tsa.seasonal_decompose(y_mf, model='additive')
fig = decomposition.plot()
#fig = decomposition1.plot()
#fig = decomposition2.plot()
#fig = decomposition3.plot()
plt.show()

In [7]:
#подбираем параметры для модели
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

Examples of parameter for SARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 12)
SARIMAX: (0, 0, 1) x (0, 1, 0, 12)
SARIMAX: (0, 1, 0) x (0, 1, 1, 12)
SARIMAX: (0, 1, 0) x (1, 0, 0, 12)


Подбор параметров и прогноз 

In [ ]:
#запускаем перебеор
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y_t2,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
        except: 
            continue

In [ ]:
mod = sm.tsa.statespace.SARIMAX(y_t2,
                                order=(0, 1, 1),
                                seasonal_order=(1, 1, 1,12 ),
                                #enforce_stationarity=False,
                                enforce_invertibility=False)
results_t2 = mod.fit()
print(results.summary().tables[1])

In [ ]:
#диагностика
results_t2.plot_diagnostics(figsize=(18, 8))

plt.show()

In [ ]:
# тест на исторических данных 
pred = results_t2.get_prediction(start=pd.to_datetime('2022-02-01'), dynamic=False)
pred_ci = pred.conf_int()
ax = y_t2['2020':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 4))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('NPS')
plt.legend()
plt.title('Tele2')
plt.show()

In [12]:
# ошибка
y_forecasted = pred.predicted_mean
y_truth = y_t2['2022-02-01':]
mse = ((y_forecasted - y_truth.Значение) ** 2).mean()
print('The Mean Squared Error is {}'.format(round(mse, 5)))
print('The Root Mean Squared Error is {}'.format(round(np.sqrt(mse), 3)))

The Mean Squared Error is 0.00017
The Root Mean Squared Error is 0.013


In [13]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_truth.Значение , y_forecasted)

0.00016896541620418026

In [ ]:
pred_uc = results.get_forecast(steps=12)
pred_ci = pred_uc.conf_int()
ax = y_t2.plot(label='observed', figsize=(14, 4))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.20)
ax.set_xlabel('Date')
ax.set_ylabel('nps')
plt.legend()
plt.show()

2022

In [ ]:
y_forecasted = pred.predicted_mean
y_forecasted.head(13)

In [ ]:
y_truth.Значение

In [ ]:
yy = y_truth.Значение
xx = y_forecasted
pr2022 = pd.concat([yy, xx], axis= 1 )
pr2022['diff'] = pr2022['Значение'] - pr2022['predicted_mean']
pr2022

In [165]:
#pr2022.to_excel('t2_forecast 2022.xlsx')

2023

In [ ]:
a = pred_ci.head(12)
a

In [ ]:
#прогноз
forecast = pred_uc.predicted_mean.head(12)
forecast = pd.DataFrame(forecast)
forecast

In [ ]:
d =a.join(forecast)
d

In [30]:
#d.to_excel('t2_2023.xlsx')